# Sales Forecasting - Capstone Project

## Problem Statement

Rossmann is a European drug distributor which operates over 3,000 drug stores across seven European countries. Since a lot of drugs come with a short shelf life, that is, they do not have a long expiry date, it becomes imperative for Rossmann to accurately forecast sales at their individual stores. Currently, the forecasting is taken care of by the store managers who are tasked with forecasting daily sales for the next six weeks. 
 
As expected, store sales are influenced by many factors, including promotional campaigns, competition, state holidays, seasonality, and locality.
 
With thousands of individual managers predicting sales based on their unique circumstances and intuitions, the accuracy of the forecasts is quite varied. To overcome this problem, the company has hired you as a data scientist to work on the forecasting problem. As part of your job role, you are tasked with building a forecasting model to forecast the daily sales for the next six weeks. To help you with the same, you have been provided with historical sales data for 1,115 Rossmann stores.

### Data definition

The data is provided in two tables, stores and train. The store table contains the metadata for every single store including the following:
 
__Store__ - an Id that represents the store<br>
__StoreType__ - differentiates between 4 different store models: a, b, c, d<br>
__Assortment__ - describes an assortment level: a = basic, b = extra, c = extended<br>
__CompetitionDistance__ - describes thedistance in meters to the nearest competitor store<br>
__CompetitionOpenSince__[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened<br>
__Promo2__ - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating<br>
__Promo2Since__[Year/Week] - describes the year and calendar week when the store started participating in Promo2<br>
__PromoInterval__ - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb, May, Aug, Nov" means each round starts in February, May, August, November of any given year for that store<br>
<br>
The train table contains the sales data for individual stores at a daily level along with the details about the day<br>
<br>
__Store__ - a unique Id for each store<br>
__DayOfWeek__ - Describes the day of the week (1 - Monday till 7 - Sunday)<br>
__Date__ - Describes the date on the day<br>
__Sales__ - the turnover for any given day (this is what you are forecasting)<br>
__Customers__ - the number of customers on a given day<br>
__Open__ - an indicator for whether the store was open: 0 = closed, 1 = open<br>
__StateHoliday__ - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None<br>
__SchoolHoliday__ - indicates if the (Store, Date) was affected by the closure of public schools<br>
<br>
Since the company is just embarking on this project, the scope has been kept to nine key stores across Europe. The stores are key for the company keeping in mind the revenue and historical prestige associated with them. These stores are numbered - 1,3,8,9,13,25,29,31 and 46.<br>
<br> 
While attempting to forecast sales, it is advisable to keep the following question in mind.<br>
<br> 
* Is the sales data non-stationary? If so, how do you find it and correct it?
* Is the data cointegrated? Which variables are cointegrated and how do you find them?
* What is the impact of the number of customers on sales?
* What is the impact of promo and promo2 variables on sales? How do you measure it?
* Forecast sales for the next 6 weeks? Report the accuracy of the model using MAPE.
<br>
<br>While you are solving the problem, the following steps can be used as a reference. Please note that you can take a different approach as well.<br>
<br> 
* Find outliers at the 99th percentile and remove them.
* Standardize the sales and number of customers variables before modelling.
* Determine if the data is stationary
* If stationary then apply Vector Autoregression Model.
* If non-stationary then specify the model in differences
* Make sales, promo2 and any other variables you think of as dependent variables.
* Check for cointegration using the Johansen test. 
* Predict sales for the next 6 weeks.

### import libraries and dependencies

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [7]:
store_data = pd.read_csv('store.csv')
sales_data = pd.read_csv('train.csv')

In [11]:
print('Store data rows %d and columns %d.' % store_data.shape)
print('Store Sales data rows %d and columns %d.' % sales_data.shape)

Store data rows 1115 and columns 10.
Store Sales data rows 1017209 and columns 9.


In [12]:
store_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [13]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


__No null values present in the sales data.__

### Converting all the dates from object to datetime

In [16]:
sales_data['Date'] = pd.to_datetime(sales_data.Date)

In [17]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  int64         
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  int64         
 6   Promo          1017209 non-null  int64         
 7   StateHoliday   1017209 non-null  object        
 8   SchoolHoliday  1017209 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 69.8+ MB
